# Import Dependencies and Read CSV

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesRegressor
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


#  Import and read the csv.
import pandas as pd 
import os

### Use on Google Colab
import io
from google.colab import files
uploaded = files.upload()
co_covid_df= pd.read_csv(io.BytesIO(uploaded['CleanCovidData.csv']))
co_covid_df.head()



Saving CleanCovidData.csv to CleanCovidData.csv


,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019,greater_than_average
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421,1
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233,0
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590,0
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029,1
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581,0


# Clean the Data

In [2]:
clean_covid_load = os.path.join("covid19_county_metrics_mod.csv")

In [3]:
#co_covid_df = pd.read_csv(clean_covid_load)
#co_covid_df

In [4]:
co_covid_df = co_covid_df.rename(columns = {'county':'changeme'})
co_covid_df

,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019,greater_than_average
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421,1
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233,0
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590,0
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029,1
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Summit County,364.3,677.0,0.117,0.117,">30,000",8117,26541,85.6,23785,76.7,A,4,Non-metro,8,31011,1
60,Teller County,270.4,670.0,0.109,0.117,"30,000 or less",8119,13250,52.2,12635,49.8,A,3,Metro,3,25388,1
61,Washington County,393.5,931.9,0.119,0.106,"30,000 or less",8121,1500,30.6,1353,27.6,B,5,Non-metro,5,4908,1
62,Weld County,444.2,844.8,0.125,0.119,">30,000",8123,187498,57.8,173708,53.5,B,8,Metro,4,324492,1


In [5]:
#co_covid_df['County'] = co_covid_df['changeme'].astype(str) + ' County'
#co_covid_df
#co_covid_df.drop(['changeme'], axis=1, inplace=True)


In [6]:
co_covid_df = co_covid_df.rename(columns = {"One-Week Cumulative Incidence":"OneWeek_Cumulative_Incidence", "Two-Week Cumulative Incidence":"TwoWeek_Cumulative_Incidence", "One-Week Average Positivity":"OneWeek_Average Positivity", "Two-Week Average Positivity":"TwoWeek_Average_Positivity", "County size":"County_size"})
co_covid_df

,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019,greater_than_average
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421,1
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233,0
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590,0
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029,1
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Summit County,364.3,677.0,0.117,0.117,">30,000",8117,26541,85.6,23785,76.7,A,4,Non-metro,8,31011,1
60,Teller County,270.4,670.0,0.109,0.117,"30,000 or less",8119,13250,52.2,12635,49.8,A,3,Metro,3,25388,1
61,Washington County,393.5,931.9,0.119,0.106,"30,000 or less",8121,1500,30.6,1353,27.6,B,5,Non-metro,5,4908,1
62,Weld County,444.2,844.8,0.125,0.119,">30,000",8123,187498,57.8,173708,53.5,B,8,Metro,4,324492,1


In [7]:
co_covid_df.dtypes

County                                object
oneweek_cumulative_incidence         float64
twoweek_cumulative_incidence         float64
oneweek_average_positivity           float64
twoweek_average_positivity           float64
county_size                           object
fips                                   int64
administered_dose1_recip               int64
administered_dose1_pop_pct           float64
series_complete_yes                    int64
series_complete_pop_pct              float64
svi_ctgy                              object
series_complete_pop_pct_svi            int64
metro_status                          object
series_complete_pop_pct_ur_equity      int64
census2019                             int64
greater_than_average                   int64
dtype: object

In [8]:
# Find metro status values
co_covid_df['metro_status'].unique

<bound method Series.unique of 0         Metro
1     Non-metro
2         Metro
3     Non-metro
4     Non-metro
        ...    
59    Non-metro
60        Metro
61    Non-metro
62        Metro
63    Non-metro
Name: metro_status, Length: 64, dtype: object>

In [9]:
# Encode metro status string values into numeric values
metro_dict = {'Non-metro': 0, 'Metro': 1}
co_covid_df = co_covid_df.replace({'metro_status': metro_dict})

In [10]:
# Find values of county size
co_covid_df['county_size'].unique

<bound method Series.unique of 0            >30,000
1     30,000 or less
2            >30,000
3     30,000 or less
4     30,000 or less
           ...      
59           >30,000
60    30,000 or less
61    30,000 or less
62           >30,000
63    30,000 or less
Name: county_size, Length: 64, dtype: object>

In [11]:
# Encode county size string values into numeric values
county_dict = {'30,000 or less': 0, '>30,000': 1}
co_covid_df = co_covid_df.replace({'county_size': county_dict})


In [12]:
# Find values of svi category
co_covid_df['svi_ctgy'].unique

<bound method Series.unique of 0     C
1     D
2     B
3     B
4     C
     ..
59    A
60    A
61    B
62    B
63    C
Name: svi_ctgy, Length: 64, dtype: object>

In [13]:
# Encode svi category string values into numeric values
svi_dict = {'A': 1, 'B': 2, 'C': 3, 'D':4}
co_covid_df = co_covid_df.replace({'svi_ctgy': svi_dict})

In [14]:
# Relevant Features 
co_covid_df = co_covid_df[['County', 'twoweek_average_positivity', 'county_size', 'series_complete_yes', 'series_complete_pop_pct', 'metro_status', 'census2019', 'greater_than_average', 'svi_ctgy']]
co_covid_df

,County,twoweek_average_positivity,county_size,series_complete_yes,series_complete_pop_pct,metro_status,census2019,greater_than_average,svi_ctgy
0,Adams County,0.101,1,309975,59.9,1,517421,1,3
1,Alamosa County,0.088,0,8162,50.3,0,16233,0,4
2,Arapahoe County,0.083,1,394045,60.0,1,656590,0,2
3,Archuleta County,0.148,0,7446,53.1,0,14029,1,2
4,Baca County,0.065,0,1322,36.9,0,3581,0,3
...,...,...,...,...,...,...,...,...,...
59,Summit County,0.117,1,23785,76.7,0,31011,1,1
60,Teller County,0.117,0,12635,49.8,1,25388,1,1
61,Washington County,0.106,0,1353,27.6,0,4908,1,2
62,Weld County,0.119,1,173708,53.5,1,324492,1,2


## Split the Data into Training and Testing 

In [15]:
# Generate the categorical variable list
feat_cat = ['County','twoweek_average_positivity', 'series_complete_yes', 'census2019', ]

In [16]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [17]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(co_covid_df[feat_cat]))

In [18]:
encode_df.columns = enc.get_feature_names(feat_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,County_Adams County,County_Alamosa County,County_Arapahoe County,County_Archuleta County,County_Baca County,County_Bent County,County_Boulder County,County_Broomfield County,County_Chaffee County,County_Cheyenne County,County_Clear Creek County,County_Conejos County,County_Costilla County,County_Crowley County,County_Custer County,County_Delta County,County_Denver County,County_Dolores County,County_Douglas County,County_Eagle County,County_El Paso County,County_Elbert County,County_Fremont County,County_Garfield County,County_Gilpin County,County_Grand County,County_Gunnison County,County_Hinsdale County,County_Huerfano County,County_Jackson County,County_Jefferson County,County_Kiowa County,County_Kit Carson County,County_La Plata County,County_Lake County,County_Larimer County,County_Las Animas County,County_Lincoln County,County_Logan County,County_Mesa County,...,census2019_8205,census2019_9700,census2019_10019,census2019_11267,census2019_12172,census2019_13283,census2019_14029,census2019_14506,census2019_15734,census2019_16233,census2019_17462,census2019_17767,census2019_18278,census2019_18845,census2019_20356,census2019_22409,census2019_25388,census2019_25638,census2019_26183,census2019_26729,census2019_29068,census2019_31011,census2019_31162,census2019_42758,census2019_47839,census2019_55127,census2019_56221,census2019_60061,census2019_70465,census2019_154210,census2019_168424,census2019_324492,census2019_326196,census2019_351154,census2019_356899,census2019_517421,census2019_582881,census2019_656590,census2019_720403,census2019_727211
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Merge one-hot encoded features and drop the originals
co_covid_df2 = co_covid_df.merge(encode_df, left_index=True, right_index=True).drop(columns=feat_cat, axis=1)
co_covid_df2

,county_size,series_complete_pop_pct,metro_status,greater_than_average,svi_ctgy,County_Adams County,County_Alamosa County,County_Arapahoe County,County_Archuleta County,County_Baca County,County_Bent County,County_Boulder County,County_Broomfield County,County_Chaffee County,County_Cheyenne County,County_Clear Creek County,County_Conejos County,County_Costilla County,County_Crowley County,County_Custer County,County_Delta County,County_Denver County,County_Dolores County,County_Douglas County,County_Eagle County,County_El Paso County,County_Elbert County,County_Fremont County,County_Garfield County,County_Gilpin County,County_Grand County,County_Gunnison County,County_Hinsdale County,County_Huerfano County,County_Jackson County,County_Jefferson County,County_Kiowa County,County_Kit Carson County,County_La Plata County,County_Lake County,...,census2019_8205,census2019_9700,census2019_10019,census2019_11267,census2019_12172,census2019_13283,census2019_14029,census2019_14506,census2019_15734,census2019_16233,census2019_17462,census2019_17767,census2019_18278,census2019_18845,census2019_20356,census2019_22409,census2019_25388,census2019_25638,census2019_26183,census2019_26729,census2019_29068,census2019_31011,census2019_31162,census2019_42758,census2019_47839,census2019_55127,census2019_56221,census2019_60061,census2019_70465,census2019_154210,census2019_168424,census2019_324492,census2019_326196,census2019_351154,census2019_356899,census2019_517421,census2019_582881,census2019_656590,census2019_720403,census2019_727211
0,1,59.9,1,1,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,50.3,0,0,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,60.0,1,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,53.1,0,1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,36.9,0,0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1,76.7,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0,49.8,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [20]:
#co_covid_df2.corr()

In [21]:
# Split our preprocessed data into our features and target arrays
y = co_covid_df2["greater_than_average"]
X = co_covid_df2.drop("greater_than_average", axis=1)

In [22]:
# Split the preprocessed data into a training and testing dataset
# The training and testing data is split into 75% and 25%, respectively.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

In [24]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [25]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Undersampling, We are using undersampling to normalize our data.

In [26]:
from collections import Counter

In [27]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 22, 1: 22})

In [28]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [29]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


array([[6, 3],
       [1, 6]])

In [30]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7619047619047619

In [31]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.67      0.86      0.75      0.76      0.56         9
          1       0.67      0.86      0.67      0.75      0.76      0.58         7

avg / total       0.77      0.75      0.77      0.75      0.76      0.57        16



# Decision Tree, We have decided to use a decision tree model for our Machine Learning Model because we are predicting a binary outcome.

In [32]:
# Import Dependencies
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [33]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [34]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

In [35]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


In [36]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [37]:
# Displaying results
print("Confusion Matrix Decsion Tree")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix Decsion Tree


,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


Accuracy Score : 0.6875
Classification Report
              precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.29      0.44         7

    accuracy                           0.69        16
   macro avg       0.82      0.64      0.61        16
weighted avg       0.80      0.69      0.63        16



# Logistic Regression, We also wanted to look at how the model performed for a Logistic Regression model because it also predicts binary values.

In [38]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [39]:
classifier.fit(X_resampled, y_resampled)

LogisticRegression(max_iter=200, random_state=1)

In [40]:
y_pred = classifier.predict(X_test_scaled)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [41]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.75


# Random Forest Model

In [42]:
# Create a random forest classifier.
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [43]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [44]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [45]:
# Calculating the confusion matrix.
cm2 = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df2 = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df2

,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


In [46]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.6875

In [47]:
# Displaying results
print("Confusion Matrix Random Forest")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix Random Forest


,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


Accuracy Score : 0.6875
Classification Report
              precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.29      0.44         7

    accuracy                           0.69        16
   macro avg       0.82      0.64      0.61        16
weighted avg       0.80      0.69      0.63        16



In [48]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.01980644, 0.09707038, 0.0332114 , 0.02647412, 0.006493  ,
       0.0052897 , 0.        , 0.0045803 , 0.        , 0.00292828,
       0.00355483, 0.00287123, 0.00298704, 0.00121866, 0.00492512,
       0.        , 0.00665667, 0.00131633, 0.00786731, 0.00862802,
       0.00753274, 0.00024132, 0.00580203, 0.00599723, 0.00342974,
       0.00976447, 0.        , 0.00243003, 0.0048595 , 0.00187912,
       0.        , 0.00561887, 0.        , 0.        , 0.00378817,
       0.        , 0.00365764, 0.        , 0.        , 0.00083686,
       0.00184073, 0.00288189, 0.00621725, 0.00613793, 0.        ,
       0.00114318, 0.00676979, 0.        , 0.0033122 , 0.        ,
       0.00182397, 0.00425501, 0.00337541, 0.00086765, 0.0024054 ,
       0.00353059, 0.00536201, 0.00160628, 0.        , 0.00544291,
       0.00486329, 0.00267977, 0.00251068, 0.00385629, 0.00470502,
       0.        , 0.        , 0.00861582, 0.0014129 , 0.00202198,
       0.00479217, 0.00256189, 0.00316821, 0.        , 0.00069

In [49]:
#co_covid_df.to_csv('CleanCountyCoviddata.csv', columns=["County", "date", "OneWeek_Cumulative_Incidence", "TwoWeek_Cumulative_Incidence", "OneWeek_Average Positivity", "TwoWeek_Average_Positivity", "County_size"])